In [37]:
import os
print os.getcwd()

/Users/eric


In [38]:
os.chdir("..")
import config

In [39]:
import twitter
from twitter.parse_tweet import Emoticons
import numpy as np
import time
import re

In [1]:
api = twitter.Api(consumer_key=config.consumer_key,
                  consumer_secret=config.consumer_secret,
                  access_token_key=config.access_token,
                  access_token_secret=config.access_token_secret, timeout=60)

NameError: name 'twitter' is not defined

In [41]:
positive_stream = api.GetStreamFilter(track=Emoticons.POSITIVE)
negative_stream = api.GetStreamFilter(track=Emoticons.NEGATIVE)

In [42]:
tweets = []
for i in range(100):
    tweets.append(positive_stream.next())
    tweets.append(negative_stream.next())

In [43]:
tweet_texts = []
for i, tweet in enumerate(tweets):
    try:
        tweet_texts.append(tweet['text'])
    except KeyError:
        print "Problem with %i" % i
text = ''.join(tweet_texts)

Problem with 117
Problem with 121
Problem with 135
Problem with 141
Problem with 182


In [44]:
class TweetEncoder(object):
    
    def __init__(self, text):
        self.chars = [''] + list(set(text))
        self.nchars = len(self.chars)
        self._c2i = {c:i for i,c in enumerate(self.chars)}

    def __call__(self, obj):
        if isinstance(obj, np.ndarray):
            if len(obj.shape) == 3:
                return self._decode_tweets(obj)
            elif len(obj.shape) == 2:
                return self._decode_tweet(obj)
        elif isinstance(obj, list):
            return self._encode_tweets(obj)
        elif isinstance(obj, unicode):
            return self._encode_tweet(obj)
        elif isinstance(obj, str):
            return self._encode_tweet(obj)
        else:
            raise ValueError
    
    def _encode_tweet(self, tweet):
        x = np.zeros((140, self.nchars))
        offset = 140 - len(tweet)
        for i, c in enumerate(tweet):
            j = self.c2i(c)
            x[i+offset, j] = 1
        return x
    
    def _encode_tweets(self, tweets):
        xs = [self._encode_tweet(t) for t in tweets]
        return np.stack(xs)
    
    def _decode_tweet(self, x):
        
        def choose_char(p):
            if np.sum(p) > 1e-6:
                p /= np.sum(p)
                return np.random.choice(self.chars, p=p)
            else:
                return ''
            
        chars = [choose_char(x[i,:]) for i in range(x.shape[0])]
        return ''.join(chars)
        
    def _decode_tweets(self, x):
        return [self._decode_tweet(x[i, :, :]) for i in range(x.shape[0])]
    
    def c2i(self, c):
        try:
            return self._c2i[c]
        except KeyError:
            return 0
    

In [45]:
encoder = TweetEncoder(text)

def data_generator(pos, neg, encoder, batch_size=100):
    
    while True:
        xs = []
        ys = []
        
        for _ in range(int(batch_size/2)):
            
            while True:
                try:
                    t = strip_emoticon(pos.next()['text'])
                    x = encoder(t)
                    xs.append(x)
                    ys.append(0.1)
                    break
                except KeyError:
                    print "Hit an error in positive data gen, pausing.."
                    time.sleep(1) # if we hit a limit with the twitter API
            
            while True:
                try:
                    t = strip_emoticon(neg.next()['text'])
                    x = encoder(t)
                    xs.append(x)
                    ys.append(0)
                    break
                except KeyError:
                    print "Hit an error in negative data gen, pausing..."
                    time.sleep(0.1)
            
        yield np.stack(xs), np.array(ys)

In [46]:
regex = '|'.join(Emoticons.NEGATIVE + Emoticons.POSITIVE)
regex = re.sub('\(', '\\\(', regex)
regex = re.sub('\)', '\\\)', regex)
regex = re.sub('\'', '\\\'', regex)
regex = re.sub('\^', '\\\^', regex)
regex = re.sub('\*', '\\\*', regex)
regex = re.sub('\[', '\\\[', regex)

In [47]:
def strip_emoticon(text):
    return re.sub(regex, "", text)

In [48]:
gen = data_generator(positive_stream, negative_stream, encoder)

In [49]:
import keras
from keras.models import Sequential
from keras.layers.recurrent import LSTM
from keras.layers.core import Dense, Dropout

In [50]:
model = Sequential()
model.add(LSTM(128, input_shape=(140, encoder.nchars), return_sequences=True))
model.add(LSTM(128, input_shape=(140, encoder.nchars), return_sequences=False))
model.add(Dense(256, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(1, activation="sigmoid"))

In [51]:
model.compile(loss="binary_crossentropy", optimizer="RMSprop")

In [52]:
model.fit_generator(gen, samples_per_epoch=1000, nb_epoch=2)

Epoch 1/2
Hit an error in negative data gen, pausing...
Hit an error in negative data gen, pausing...
Hit an error in negative data gen, pausing...
Hit an error in negative data gen, pausing...
Hit an error in negative data gen, pausing...
Hit an error in negative data gen, pausing...
Hit an error in negative data gen, pausing...
Hit an error in negative data gen, pausing...
Hit an error in negative data gen, pausing...
Hit an error in positive data gen, pausing..
 100/1000 [==>...........................] - ETA: 18s - loss: 0.7155Hit an error in negative data gen, pausing...
Hit an error in negative data gen, pausing...
Hit an error in negative data gen, pausing...
Hit an error in positive data gen, pausing..
 200/1000 [=====>........................] - ETA: 14s - loss: 0.6804Hit an error in negative data gen, pausing...
Hit an error in negative data gen, pausing...
Hit an error in negative data gen, pausing...
Hit an error in negative data gen, pausing...
Hit an error in negative dat

Exception in thread Thread-5:
Traceback (most recent call last):
  File "/System/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/threading.py", line 810, in __bootstrap_inner
    self.run()
  File "/System/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/threading.py", line 763, in run
    self.__target(*self.__args, **self.__kwargs)
  File "/Users/eric/Projects/sentiment/venv/lib/python2.7/site-packages/keras/engine/training.py", line 425, in data_generator_task
    generator_output = next(generator)
  File "<ipython-input-45-de2b133fde86>", line 24, in data_generator
    t = strip_emoticon(neg.next()['text'])
  File "/Users/eric/Projects/sentiment/venv/lib/python2.7/site-packages/twitter/api.py", line 4428, in GetStreamFilter
    for line in resp.iter_lines():
  File "/Users/eric/Projects/sentiment/venv/lib/python2.7/site-packages/requests/models.py", line 720, in iter_lines
    for chunk in self.iter_content(chunk_size=chunk_size, decode_unicode=decode_u

 700/1000 [====================>.........] - ETA: 4s - loss: 0.3505

Exception: output of generator should be a tuple (x, y, sample_weight) or (x, y). Found: None

In [2]:
re

NameError: name 're' is not defined

In [3]:
import re

In [44]:
r = re.search('[0-9]+[a-z]', '1234h')

In [45]:
r.group(0)

'1234h'